In [ ]:
%%capture
!pip install datasets evaluate nltk jiwer rouge_score accelerate transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoConfig, AutoProcessor, Qwen2VLForConditionalGeneration, TrainingArguments, Trainer
import torch

In [ ]:
# processor = AutoProcessor.from_pretrained("prithivMLmods/Qwen2-VL-OCR-2B-Instruct")
# model = Qwen2VLForConditionalGeneration.from_pretrained("prithivMLmods/Qwen2-VL-OCR-2B-Instruct")
# device = "cuda" if torch.cuda.is_available() else "cpu"
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
#             {"type": "text", "text": "What animal is on the candy?"}
#         ]
#     },
# ]
# inputs = processor.apply_chat_template(
# 	messages,
# 	add_generation_prompt=True,
# 	tokenize=True,
# 	return_dict=True,
# 	return_tensors="pt",
# ).to(device)

In [ ]:
# print(processor.image_processor.size.keys())
# longest_edge = processor.image_processor.size["longest_edge"]
# shortest_edge = processor.image_processor.size["shortest_edge"]
# print(f"Processor longest_edge: {longest_edge}, shortest_edge: {shortest_edge}")

dict_keys(['longest_edge', 'shortest_edge'])
Processor longest_edge: 12845056, shortest_edge: 3136


In [ ]:
from datasets import load_dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union, Optional
from PIL import Image
from transformers.tokenization_utils_base import BatchEncoding
import torch # Import torch

# Load the dataset
dataset = load_dataset("OleehyO/latex-formulas", "cleaned_formulas")

# dataset = load_dataset("Manu1309/latex_ocr_with_categories")

file_names = []
texts = []
#categories = []
for example in dataset["train"]:
  texts.append(example["text"])
  file_names.append(str(example['image']))
  #categories.append(example['category'])

In [ ]:
train_val_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_ds = train_val_split["train"]
val_test_split = train_val_split["test"].train_test_split(test_size=0.5, seed=42)
val_ds = val_test_split["train"]
test_ds = val_test_split["test"]

{'longest_edge': 12845056, 'shortest_edge': 3136}

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict({"image": file_names, "texts": texts})
print(df.shape)

df.head()
df['texts'].unique()

(68686, 3)


array(['{ \\frac { N } { M } } \\in { \\bf Z } , { \\frac { M } { P } } \\in { \\bf Z } , { \\frac { P } { Q } } \\in { \\bf Z }',
       'D _ { \\mu } ^ { \\alpha \\beta } \\bar { A } _ { \\mu } ^ { \\alpha \\beta } = 0 ,',
       'H ^ { \\prime } = \\beta N \\int d \\lambda \\biggl \\{ \\frac { 1 } { 2 \\beta ^ { 2 } N ^ { 2 } } \\partial _ { \\lambda } \\zeta ^ { \\dagger } \\partial _ { \\lambda } \\zeta + V ( \\lambda ) \\zeta ^ { \\dagger } \\zeta \\biggr \\} \\ .',
       ...,
       'v _ { p q } \\equiv \\frac { p \\cdot q } { \\sqrt { \\left( p \\cdot q \\right) ^ { 2 } - p ^ { 2 } q ^ { 2 } } } ,',
       'A _ { i } ^ { h } = - i \\partial _ { i } h * h _ { * } ^ { - 1 } + h * A _ { i } * h _ { * } ^ { - 1 } .',
       'V = W _ { i ^ { * } } ^ { * } K ^ { - 1 { i ^ { * } } j } W _ { j } ,'],
      dtype=object)

In [ ]:
from datasets import ClassLabel

ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)
def map_text2id(example):
    example['text'] = ClassLabels.str2int(example['text'])
    return example

In [ ]:
def preprocess_function(examples):
    # Access the image and text for a single example
    image = examples["image"]
    text = examples["text"]

    # Ensure image is in RGB format before processing
    image_rgb = image.convert("RGB")

    # Process the image and text. The processor should return tensors when return_tensors="pt"
    # The Qwen2VL processor expects images as a list
    inputs = processor(text=text, images=[image_rgb], return_tensors="pt", padding=True, truncation=True)

    # Tokenize the labels (text)
    labels = processor.tokenizer(text, return_tensors="pt", padding=True, truncation=True)["input_ids"]

    # Ensure the output structure matches what the data collator expects
    processed_inputs = {
        "input_ids": inputs["input_ids"].squeeze(0),
        "attention_mask": inputs["attention_mask"].squeeze(0),
        "pixel_values": inputs["pixel_values"].squeeze(0), # Ensure this is a tensor
        "image_grid_thw": inputs["image_grid_thw"].squeeze(0), # Ensure this is a tensor
        "labels": labels.squeeze(0), # Ensure this is a tensor
    }

    return processed_inputs

# Apply the preprocessing function to the entire dataset with batched=False
# The new dataset has 'train' and 'test' splits
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=False, # Changed to False to process individually
    remove_columns=["image", "text"] # Remove original columns after processing
)


# Set the dataset format to 'torch'
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw', 'labels'])

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, stratify_by_column="category")

## Define training arguments

### Subtask:
Set up the training parameters such as the output directory, learning rate, number of epochs, and other relevant settings using `TrainingArguments`.


In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen2-vl-ocr-finetuned",  # Output directory
    learning_rate=5e-5,  # Learning rate
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=2,  # Batch size per device for training
    per_device_eval_batch_size=2,  # Batch size per device for evaluation
    weight_decay=0.01,  # Weight decay
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    push_to_hub=False, # Do not push to hub during training
    report_to="none", # Do not report to any service
    dataloader_num_workers=4,
)

In [ ]:
data_collator = DataCollatorForVision2Seq(processor=processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

## Start training

### Subtask:
Begin the fine-tuning process by calling the `train()` method on the initialized Trainer.


In [ ]:
# Accessing pixel_values from the first example in the training dataset
first_example_pixel_values = tokenized_dataset["train"][0]["pixel_values"]
print(first_example_pixel_values)

tensor([[ 1.9303,  1.9303,  1.9303,  ..., -0.1151, -0.1151, -0.1151],
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459, -0.5701],
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459],
        ...,
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459],
        [ 1.9303,  1.9157,  1.9157,  ...,  2.1459,  2.1459,  2.1459],
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459]])


In [ ]:
trainer.train()

In [ ]:
from datasets import load_dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union, Optional
from PIL import Image
from transformers.tokenization_utils_base import BatchEncoding
import torch

# Load the dataset
dataset = load_dataset("OleehyO/latex-formulas", "cleaned_formulas")

file_names = []
texts = []
#categories = []
for example in dataset["train"]:
  texts.append(example["text"])
  file_names.append(str(example['image']))
  #categories.append(example['category'])

In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor
import torch
from PIL import Image

# load model, tokenizer, and feature extractor
model = VisionEncoderDecoderModel.from_pretrained("DGurgurov/im2latex")
tokenizer = AutoTokenizer.from_pretrained("DGurgurov/im2latex")
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k") # using the original feature extractor for now

# prepare an image
image = Image.open("path/to/your/image.png")
pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values

# generate LaTeX formula
generated_ids = model.generate(pixel_values)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print("Generated LaTeX formula:", generated_texts[0])


# Task
Train and evaluate the `DGurgurov/im2latex` model on the "OleehyO/latex-formulas" dataset.

## Load model and processor

### Subtask:
Load the `DGurgurov/im2latex` model, its tokenizer, and feature extractor.


**Reasoning**:
Load the pre-trained model, tokenizer, and feature extractor for the im2latex model.



In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor

model = VisionEncoderDecoderModel.from_pretrained("DGurgurov/im2latex")
tokenizer = AutoTokenizer.from_pretrained("DGurgurov/im2latex")
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/962M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


## Prepare dataset

### Subtask:
Adapt the existing dataset loading and preprocessing steps to work with the new tokenizer and feature extractor. This includes handling image and text data appropriately for the `VisionEncoderDecoderModel`.


**Reasoning**:
Implement the data preprocessing function and apply it to the dataset, then split the dataset and set the format.



In [ ]:
print(type(dataset["train"][0]['image']))
print(dataset["train"][0]['image'])
print(type(dataset["train"][0]['latex_formula']))
print(dataset["train"][0]['latex_formula'])

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=323x44 at 0x7D0C55565E10>
<class 'str'>
\begin{align*}-\div (y^{1-2m}\nabla w)=0\mbox{in}\mathbb R^n\times\mathbb R_+;w\big|_{y=0}=|u|\end{align*}


**Reasoning**:
The output from the previous step shows that the 'image' column in the dataset already contains PIL Image objects, not file paths. The previous `preprocess_dataset` function was trying to call `Image.open()` on these Image objects, which caused the `AttributeError: read`. I need to modify the `preprocess_dataset` function to directly use the PIL Image objects from the 'image' column instead of trying to open them from a path.



In [ ]:
from datasets import load_dataset
from PIL import Image

# Load the dataset
dataset = load_dataset("OleehyO/latex-formulas", "cleaned_formulas")

def preprocess_dataset(examples):
    # examples['image'] is already a list of PIL Image objects when batched=True
    images = [img.convert("RGB") for img in examples['image']]
    pixel_values = feature_extractor(images=images, return_tensors="pt", padding=True).pixel_values
    labels = tokenizer(examples['latex_formula'], return_tensors="pt", padding=True, truncation=True).input_ids
    return {"pixel_values": pixel_values, "labels": labels}

# Apply the preprocessing function
processed_dataset = dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=["image", "latex_formula"]
)

# Split the dataset
train_test_split = processed_dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# Set the format
train_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])
test_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

print(train_dataset)
print(test_dataset)

## Define Training Arguments

### Subtask:
Set up the training parameters, similar to the previous model, using `TrainingArguments` but adjusting for the new model architecture if necessary.

**Reasoning**:
Define the training arguments for the `VisionEncoderDecoderModel`. These arguments will configure the training process, including the output directory, learning rate, number of epochs, batch sizes, and evaluation strategy.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./im2latex-finetuned",  # Output directory
    learning_rate=5e-5,  # Learning rate
    per_device_train_batch_size=16,  # Batch size per device for training
    per_device_eval_batch_size=16,  # Batch size per device for evaluation
    num_train_epochs=3,  # Number of training epochs
    weight_decay=0.01,  # Weight decay
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    push_to_hub=False, # Do not push to hub during training
    report_to="none", # Do not report to any service
    dataloader_num_workers=4,
    predict_with_generate=True, # Use generate to compute metrics
    generation_max_length=128, # Maximum length for generated text
)

## Define Data Collator

### Subtask:
Create a data collator suitable for the `VisionEncoderDecoderModel`.

**Reasoning**:
Define a data collator that will be used by the Trainer to prepare batches of data. This data collator will handle padding of both pixel values and labels.

In [ ]:
from transformers import DataCollatorWithPadding

# Define the data collator
# The VisionEncoderDecoderModel does not have a specific data collator,
# so we use a general DataCollatorWithPadding for the labels.
# The pixel_values are already padded by the feature_extractor.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Initialize Trainer

### Subtask:
Set up the `Trainer` with the new model, training arguments, and the prepared dataset.

**Reasoning**:
Initialize the `Trainer` class with the loaded model, defined training arguments, and the preprocessed training and test datasets. The data collator is also passed to handle batching.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

## Train the Model

### Subtask:
Start the training process.

**Reasoning**:
Call the `train()` method on the initialized Trainer object to begin the fine-tuning process.

In [ ]:
trainer.train()

## Evaluate the Model

### Subtask:
Evaluate the trained model on the test dataset.

**Reasoning**:
After training, evaluate the model's performance on the test dataset using the `evaluate()` method of the Trainer.

In [ ]:
trainer.evaluate()